# Preparação dos dados

#### 1 - Carga dos dados no dataframe

In [73]:
import pandas as pd
import numpy as np
from bleach.sanitizer import Cleaner
import regex as re
import html

In [74]:
# Realizando parse de data em coluna única no momento da carga dos dados no dataframe
df = pd.read_excel("../../Data/Raw/informacao.xlsx", usecols =['objeto','texto','data_inclusao','topico'],dtype={'objeto':str, 'texto':str, 'topico':str}, parse_dates=['data_inclusao']) 

df.head()

,objeto,texto,data_inclusao,topico
0,Despacho nº 1665 que libera a unidade gerador...,Despacho nº 1665 que libera a unidade gerador...,2007-05-29 00:00:00,[BDP] - LEGADO
1,Despacho No. 1.694 da SFG que libera a UG1 (40...,Despacho No. 1.694 da SFG que libera a UG1 (40...,2007-05-31 00:00:00,[BDP] - LEGADO
2,Despacho nº 1731 que libera a unidade gerador...,Despacho nº 1731 que libera a unidade gerador...,2007-06-01 00:00:00,[BDP] - LEGADO
3,Portaria MME No. 100/2007 publicada no DOU de...,Portaria MME No. 100/2007 publicada no DOU de...,2007-06-01 00:00:00,[BDP] - LEGADO
4,Despacho nº 1617 que libera a unidade gerador...,Despacho nº 1617 que libera a unidade gerador...,2007-06-01 00:00:00,[BDP] - LEGADO


In [75]:
df.tail()

,objeto,texto,data_inclusao,topico
39967,DESPACHO ANEEL Nº 733 DE 22 DE MARÇO DE 2022,"<p style=""text-align:justify"">O DIRETOR-GERAL ...",2022-03-28 00:00:00,Receita Anual Permitida - RAP
39968,DESPACHO ANEEL Nº 733 DE 22 DE MARÇO DE 2022,"<p style=""text-align:justify"">O DIRETOR-GERAL ...",2022-03-28 00:00:00,Recurso Interposto
39969,DESPACHO ANEEL Nº 733 DE 22 DE MARÇO DE 2022,"<p style=""text-align:justify"">O DIRETOR-GERAL ...",2022-03-28 00:00:00,Ampliação reforço ou melhoria
39970,DESPACHO ANEEL Nº 736 DE 22 DE MARÇO DE 2022,"<p style=""text-align:justify"">O DIRETOR DA AG&...",2022-03-28 00:00:00,Recurso Interposto
39971,DESPACHO ANEEL Nº 736 DE 22 DE MARÇO DE 2022,"<p style=""text-align:justify"">O DIRETOR DA AG&...",2022-03-28 00:00:00,Outorga


#### 2 - Verificações

In [76]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39972 entries, 0 to 39971
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   objeto         39899 non-null  object
 1   texto          39899 non-null  object
 2   data_inclusao  39827 non-null  object
 3   topico         39827 non-null  object
dtypes: object(4)
memory usage: 1.2+ MB


#### 2 - Limpeza

Verificadas tags html e codificação de url nos textos. Provavelemente em algum momento a aplicação passou a fornecer a funcionalidade de texto rico para a edição de textos.

In [77]:
# Exemplo
df.iloc[25000]['texto']

'<p style="text-align:justify">O SUPERINTENDENTE DE REGULA&Ccedil;&Atilde;O DOS SERVI&Ccedil;OS DE GERA&Ccedil;&Atilde;O ADJUNTO DA AG&Ecirc;NCIA NACIONAL DE ENERGIA EL&Eacute;TRICA &ndash; ANEEL  no uso das atribui&ccedil;&otilde;es delegadas por meio da Portaria n&ordm; 4.163  de 30 de agosto de 2016  e tendo em vista o que consta no Processo n&ordm; 48500.004084/2016-11  decide: (i) conhecer e  no m&eacute;rito  dar provimento &agrave; solicita&ccedil;&atilde;o da Termopernambuco S.A. para autorizar a utiliza&ccedil;&atilde;o do Custo Vari&aacute;vel Unit&aacute;rio &ndash; CVU da Usina Termel&eacute;trica &ndash; UTE Termopernambuco (C&oacute;digo CEG: UTE.GN.PE.028031-3.01)  no valor de R$ 126 17/MWh (cento e vinte e seis reais e dezessete centavos por megawatt-hora)  a ser aplicado pelo Operador Nacional do Sistema El&eacute;trico &ndash; ONS a partir da primeira revis&atilde;o do Programa Mensal de Opera&ccedil;&atilde;o &ndash; PMO ap&oacute;s a publica&ccedil;&atilde;o do Desp

In [78]:
# Força conversão de coluna de texto para string evitando erros de conversão posteriores
df['texto'] = df['texto'].astype('str') 

In [79]:
# Codifica textos. Ex: Transforma AG&Ecirc;NCIA em AGÊNCIA
df['texto'] = df['texto'].apply(lambda x: html.unescape(x))

In [80]:
cleaner = Cleaner(tags=[], attributes=[], styles=[], protocols=[], strip=True, strip_comments=True, filters=None)

# Retira todas as tags html dos textos
df['texto'] = df['texto'].apply(lambda x: cleaner.clean(x))

In [81]:
# Resultado final, texto tratado

df.iloc[25000]['texto']

'O SUPERINTENDENTE DE REGULAÇÃO DOS SERVIÇOS DE GERAÇÃO ADJUNTO DA AGÊNCIA NACIONAL DE ENERGIA ELÉTRICA – ANEEL  no uso das atribuições delegadas por meio da Portaria nº 4.163  de 30 de agosto de 2016  e tendo em vista o que consta no Processo nº 48500.004084/2016-11  decide: (i) conhecer e  no mérito  dar provimento à solicitação da Termopernambuco S.A. para autorizar a utilização do Custo Variável Unitário – CVU da Usina Termelétrica – UTE Termopernambuco (Código CEG: UTE.GN.PE.028031-3.01)  no valor de R$ 126 17/MWh (cento e vinte e seis reais e dezessete centavos por megawatt-hora)  a ser aplicado pelo Operador Nacional do Sistema Elétrico – ONS a partir da primeira revisão do Programa Mensal de Operação – PMO após a publicação do Despacho; e (ii) determinar à Câmara de Comercialização de Energia Elétrica – CCEE a utilização do valor do CVU indicado no item “i” para fins de contabilização da geração verificada na UTE Termopernambuco a partir do mês de dezembro de 2019.  FELIPE ALVE

### 3 - Gravação do dataset preparado para a análise e modelagem

In [82]:
PARQUET_TEXTO_PREPARADO = '../../Data/Processed/informacoes_preparadas.parquet'

df.to_parquet(PARQUET_TEXTO_PREPARADO)